In [27]:
import json 

def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)
    
def save_json(filename, obj):
    with open(filename, "w+", encoding="utf-8") as f:
        return json.dump(obj, f, ensure_ascii=False, indent=4) 
    
def load_file(filename):
    return list(open(filename, encoding="utf-8").readlines())


def del_none(d):
    for key, value in list(d.items()):
        if value is None or value == "":
            del d[key]
        elif isinstance(value, dict):
            del_none(value)
    return d

In [2]:
event_types = load_json("../data/unified.json")["types"]

In [91]:
tut = load_json("../data/tutby.json")
cd = load_json("../data/citydog.json")
relax = load_json("../data/relax.json")
old_events = load_json("../data/events.json")

events = []
events.extend(tut)
events.extend(cd)
events.extend(relax)
events.extend(old_events)

In [92]:
tg_vk_events = []
for e in load_file("../labeled_text"):
    labeled_event = json.loads(e)
    delabled_event = {"text": labeled_event["text"], "labels": []}
    for start, end, label in labeled_event["labels"]:
        delabled_event["labels"].append((labeled_event["text"][start:end], label))
    tg_vk_events.append(delabled_event)

In [93]:
import re
import urlextract
url_extractor = urlextract.URLExtract()
        
freq = [
    "работая с артистами, которых ранее никому не удавалось привезти в Беларусь -",
    "Минские организаторы, не перестают радовать Белорусских фанатов,",
    "Минские организаторы, не перестают радовать фанатов,",
    "Baby Tape, Boulevard Depo, Loqiemean, Jeembo &",
    "Билеты можно приобрести онлайн на AFISHA.TUT.BY",
    "Билеты можно купить онлайн на AFISHA.TUT.BY",
    "Фото и видео",
    "Depo, Loqiemean, Jeembo &",
    "на AFISHA.TUT.BY Фото и видео",
    "О событии"
]

space_re = re.compile(r"[\s|\u2800]+")
comma_re = re.compile(r"(([^\d])\s*,\s*(\w))")
comma_re_2 = re.compile(r"((\w)\s*,\s*([^\d]))")
dash_re = re.compile(r"(([^\d])\s*\-+\s*([^\d]))")
underscore_re = re.compile(r"\_+")
hashtag_re = re.compile(r"#\w+")


def remove_urls(text):
    urls = list(set(url_extractor.find_urls(text)))
    urls.sort(key=lambda u: len(u), reverse=True)
    for url in urls:
        text = text.replace(url, "")
    return text

def preprocess(text):
    text = text.replace("\n", " ")
    text = remove_urls(text)
    text = hashtag_re.sub("", text)
    text = underscore_re.sub(" ", text)
    for f in freq:
        text = text.replace(f, "")
    for whole, first, second in reversed(comma_re.findall(text)):
        text = text.replace(whole, "%s, %s" % (first, second))
    for whole, first, second in reversed(comma_re_2.findall(text)):
        text = text.replace(whole, "%s, %s" % (first, second))
    text = space_re.sub(" ", text)
    return text.strip()

In [94]:
for e in events:
    text = e['description']
    e['description'] = preprocess(text)

In [95]:
def detect_lang(text):
    by_letters = ['і', 'ў', 'шч']
    for by_letter in by_letters:
        if by_letter in text:
            return "by"
    return "ru"

def is_suitable_text(text):
    if detect_lang(text) == "by":
        return False
    if len(text) > 2000 or len(text) < 80:
        return False
    if all(not c.isdigit() for c in text):
        return False
    return True

In [96]:
events = [e for e in events if is_suitable_text(e['description'])]

In [97]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [98]:
import numpy as np

def remove_duplicated_texts(evs):
    removed_idx = set()
    unique_events = []
    for i, current_text in enumerate(evs):
        if i in removed_idx:
            continue
            
        duplicates = [current_text]
        for j, text in enumerate(evs[i + 1:]):
            if j in removed_idx:
                continue
            res = similar(current_text, text)
            if res > 0.6:
                duplicates.append(text)
                removed_idx.add(j)
                
        unique_event = max(duplicates, key=lambda e: len(e))
        unique_events.append(unique_event)
        
    return unique_events

def remove_duplicated(evs):
    removed_idx = set()
    unique_events = []
    for i, current_event in enumerate(evs):
        if i in removed_idx:
            continue
            
        duplicates = [current_event]
        for j, event in enumerate(evs[i + 1:]):
            if j in removed_idx:
                continue
            res = similar(current_event['description'], event['description'])
            if res > 0.6:
                duplicates.append(event)
                removed_idx.add(j)
                
        unique_event = max(duplicates, key=lambda e: len(e['description']))
        unique_events.append(unique_event)
        
    return unique_events

In [99]:
import random
import datetime

events_by_dates = {}
for event in events:
    date = event['dates'][0]
    if 'start' in date:
        date = date['start']
    if date:
        if "hour" in date:
            dt = datetime.datetime(year=date['year'], month=date['month'], day=date['day'], hour=date['hour'], minute=date['minute'])
        else:
            dt = datetime.date(year=date['year'], month=date['month'], day=date['day'])
    else:
        dt = datetime.date(year=datetime.MINYEAR, month=1, day=1)

    if dt not in events_by_dates:
        events_by_dates[dt] = []
    events_by_dates[dt].append(event)

In [100]:
filtered_events = []
for dt, grouped_events in events_by_dates.items():
    initial_len = len(grouped_events)
    if initial_len > 1:
        grouped_events = remove_duplicated(grouped_events)
    filtered_events.extend(grouped_events)

In [101]:
import random

random.shuffle(filtered_events)
len(filtered_events)

1495

In [102]:
events_by_types = {}
for event in filtered_events:
    tags = event['tags']
    for tag in tags:
        if tag not in event_types:
            continue
        
        if tag not in events_by_types:
            events_by_types[tag] = []
        
        if len(events_by_types[tag]) > 200:
            continue
        events_by_types[tag].append(event)

            
reduced_events = []
for _, grouped_event in events_by_types.items():
    reduced_events.extend(grouped_event)
random.shuffle(reduced_events)
len(reduced_events)

852

In [103]:
for item in reduced_events:
    if "dates" in item:
        del item["dates"]
    if "poster" in item:
        del item["poster"]
    if "url" in item:
        del item["url"]
    del_none(item)

In [104]:
save_json("../data/reduced_events.json", reduced_events)

In [105]:
for event in tg_vk_events:
    event["text"] = preprocess(event["text"])
    
tg_vk_events = [e for e in tg_vk_events if is_suitable_text(e["text"])]

In [106]:
len(tg_vk_events)

1010

In [107]:
tg_vk_event_pairs = [(e, e["text"][:50]) for e in tg_vk_events]
short = [p[1] for p in tg_vk_event_pairs]
short = set(remove_duplicated_texts(short))
tg_vk_events = [e for e, short_e in tg_vk_event_pairs if short_e in short]

In [108]:
save_json("../data/reduced_events_social.json", tg_vk_events)

In [166]:
with open("labeled_text", "w+", encoding="utf-8") as f:
    for event in tg_vk_events:
        text = event["text"]
        labeled = {"text": text, "labels": []}
        for string, label in event["labels"]:
            start = text.find(string)
            if start < 0:
                continue
            end = start + len(string)
            labeled["labels"].append((start, end, label))
        f.write(json.dumps(labeled, ensure_ascii=False) + "\n")

In [116]:
def count_n_grams(n, texts):
    n_grams = {}
    for text in texts:
        for line in text.split('\n'):
            tokens = line.split(' ')
            for i in range(len(tokens) - n + 1):
                n_gram = " ".join(tokens[i:i + n])
                if n_gram not in n_grams:
                    n_grams[n_gram] = 0
                n_grams[n_gram] += 1
    return n_grams

def get_n_grams(n, text):
    tokens = text.split(' ')
    for i in range(len(tokens) - n + 1):
        n_gram = " ".join(tokens[i:i + n])
        yield n_gram

def get_top_n(n_grams, top):
    return sorted([(g, c) for g, c in n_grams.items()], key=lambda p: p[1], reverse=True)[:top]

def combine_tokens(grams):
    grams = list(grams)
    while len(grams) > 0:
        g = grams.pop()
        toks = g.split(' ')
        found = False
        for i, gr in enumerate(grams):
            gr_toks = gr.split(' ')
            if toks[len(toks) - 3:] == gr_toks[:3]:
                common = toks
                common.extend(gr_toks[3:])
                grams.append(" ".join(common))
                del grams[i]
                found = True
                break
            elif toks[:3] == gr_toks[len(gr_toks) - 3:]:
                common = gr_toks
                common.extend(toks[3:])
                grams.append(" ".join(common))
                del grams[i]
                found = True
                break
        if not found:
            yield g
            
def analyze_texts(texts):
    top_20_tri = get_top_n(count_n_grams(5, texts), 200)
    buckets = {}
    for g, c in top_20_tri:
        if c not in buckets:
            buckets[c] = []
        buckets[c].append(g)

    for count, grams in buckets.items():
        if count < 20:
            continue
        combined = list(combine_tokens(grams))
        print(count)
        for gram in sorted(combined, key=lambda p: len(p), reverse=True):
            if len(gram.split(' ')) < 6:
                continue
            print(gram)

In [161]:
import difflib

def find_place(text, place):
    place_grams = len(place.split(' '))
    matches = []
    for n in range(1, place_grams * 2):
        grams = list(get_n_grams(n, text))
        for gram in grams:
            rate = similar(place, gram)
            if rate > 0.8:
                matches.append((gram, rate))
    if not matches:
        return None
    match = max(matches, key=lambda p: p[1])[0]
    return match

In [162]:
for event in reduced_events:
    text = event["description"]
    title = event["title"]
    match = find_place(text, title)
    if match:
        print(title)
        print(match)
        print()

Деревянное и каркасное домостроение. Баня-2020
«Деревянное и каркасное домостроение. Баня-2020»

Okami Anime Party: white
Okami Anime Party: white

Karaoke
Karaoke

Стас Михайлов
Стас Михайлов

Как есть?
Как есть!

FSP Freaky Summer Party
Freaky Summer Party

Kostya Outta & QDN
Kostya Outta DJ QDN

Про Федота-стрельца
«Про Федота-стрельца»

Хит-коллекция "Территории мюзикла". 90-е
«Хит-коллекция «Территории мюзикла».

Ilya Rise & QDN
Ilya Rise - DJ QDN

Открытые экскурсии по выставке «Джаз! Коты! Весна!»
открытые экскурсии по выставке «Джаз! Коты! Весна!»,

Вода и тепло
вода и тепло

''Я вам не скажу за всю Одессу'' Леонид Утесов: вспоминая великого артиста
«Я вам не скажу за всю Одессу…» Леонид Утёсов: вспоминая великого артиста

Наследники Великой Победы
«Наследники Великой Победы»

Belphegor
Belphegor

Гараж
«Гараж»

Как есть?
Как есть!

Бой Подушками
«Бой подушками»

Проект «Lounge Night»
Проект «Lounge Night»

Пес Белого острова
«Пёс Белого острова»

Крис Норман
Крис Норман

«Тает

Семинар «Развитие кадрового потенциала банковского сектора в условиях цифровой трансформации»
Семинар «Развитие кадрового потенциала банковского сектора в условиях цифровой трансформации»

Viva Braslav Open Air 2020
Viva Braslav Open Air —

Ансамбль Радзiмiчы
Ансамбль «Радзiмiчы»

Мастер-класс «Xiaomi Day»
Мастер-класс "Xiaomi Day"

Эрика Лундмоен
Эрика Лундмоен

Игорь Христенко
Игоря Христенко.

Бой Подушками
«Бой подушками»

Концерт классической музыки "Струнные в Городе" 
концерт классической музыки. «Струнные в Городе» -

Олег Алмазов с моноспектаклем "Разговор в разговоре"
Олег Алмазов с моноспектаклем «Разговор в разговоре»

Stereo Weekend с Новым Радио
Stereo Weekend с Новым Радио

Jam Session
Jam Session!

Караоке в Бруклине
Караоке в Бруклине

Свадьба в Малиновке 
"Свадьба в Малиновке"

Виртуозы Москвы
«Виртуозы Москвы»

Анна Снегина
«Анна Снегина»

Ночь соблазнов
Ночь соблазнов

The Retuses
The Retuses

Танцевальный шоу-дуэт Сапфир
танцевальный шоу-дуэт «Сапфир»,

Colisium Mi

Красный день календаря
«Красный день календаря».

Нас поймут через 100 лет. Лазарь Хидекель
«Нас поймут через 100 лет. Лазарь Хидекель»

Открытые экскурсии по выставке «Джаз! Коты! Весна!»
открытые экскурсии по выставке «Джаз! Коты! Весна!»,

Скейт-музей СССР
Скейт-музея СССР

MLK+
MLK+

Музыка кино: Национальный академический народный оркестр Республики Беларусь им. И.Жиновича
«Музыка кино» в исполнении Национального академического народного оркестра Республики Беларусь им.И.Жиновича

Концеренция «Цифровой банкинг 2020»
Конференции «Цифровой банкинг 2020»

От винта!
«От винта!»

Диско 90х
«Диско 90-х»

«Тает LED»
«Тает LED»

Вода и тепло
вода и тепло

Воробьиная ночь
«Воробьиная ночь»

Стендап в Кубе
Стендап в Кубе

Postscriptum: Восточные рабочие в Третьем рейхе
«Postscriptum: Восточные рабочие в Третьем рейхе».

Студия «Квартал 95»
Студия «Квартал 95»

Рита Дакота
Риты Дакоты

Территория страсти
«Территория страсти»

Мэвл
Мэвл

Реанимация
Реанимация

Клава Кока
Клава Кока

Bondage F

In [154]:
reduced_events[163]

{'title': 'Оскар и Розовая Дама',
 'description': 'История о Мальчике и Розовой Даме - это история о двух людях, рассказ о двух мирах. Никто не знает, где в нём заканчивается правда и начинается фантазия, потому что жизнь сплетается в сложный клубок реальных трагедий и вымышленных образов. Как можно в 10 лет решить то, что многим не под силу в 50? Как можно подарить утешение и надежду на жизнь, когда ты стоишь на пороге неизбежного? Это рассказ о жизни и смерти, правде и лжи, отчаянии и надежде. История о каждом из нас, о том, что важно и ценно.',
 'place': {'name': 'Театр им. М. Горького',
  'address': 'г. Минск, ул. Володарского, 5',
  'url': 'https://afisha.tut.by/place/teatr-gorkogo/'},
 'tags': ['Спектакль']}